In [7]:
import normflows as nf
import torch

In [3]:
base = nf.distributions.base.DiagGaussian(2)

In [4]:
# Define list of flows
num_layers = 32
flows = []
for i in range(num_layers):
    # Neural network with two hidden layers having 64 units each
    # Last layer is initialized by zeros making training more stable
    param_map = nf.nets.MLP([1, 64, 64, 2], init_zeros=True)
    # Add flow layer
    flows.append(nf.flows.AffineCouplingBlock(param_map))
    # Swap dimensions
    flows.append(nf.flows.Permute(2, mode='swap'))

In [5]:
# If the target density is not given
model = nf.NormalizingFlow(base, flows)

# If the target density is given
target = nf.distributions.target.TwoMoons()
model = nf.NormalizingFlow(base, flows, target)

In [14]:
model.log_prob(torch.tensor([[1.0,1.0]]))

tensor([-2.8379], grad_fn=<AddBackward0>)

In [16]:
model.sample(1)

(tensor([[-1.0060, -0.6545]], grad_fn=<CatBackward0>),
 tensor([-2.5581], grad_fn=<SubBackward0>))

In [6]:
# When doing maximum likelihood learning, i.e. minimizing the forward KLD
# with no target distribution given
loss = model.forward_kld(x)

# When minimizing the reverse KLD based on the given target distribution
loss = model.reverse_kld(num_samples=512)

# Optimization as usual
loss.backward()
optimizer.step()

NameError: name 'x' is not defined